## Agglomerative Hierarchy Algorithm Implementation

In [1]:
# Add Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import numpy as np
import heapq
from scipy.spatial.distance import euclidean
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering as SklearnAgglomerative
from scipy.spatial.distance import euclidean
from sklearn.metrics import (
    silhouette_score, davies_bouldin_score, calinski_harabasz_score,
    adjusted_rand_score, homogeneity_score, completeness_score, v_measure_score
)
from itertools import product





In [2]:
dataset_path = "./../datasets"

iris_dataset_path = dataset_path + "/iris.csv"                                         
ai_global_index_path = dataset_path + "/AI_index_db.csv"
global_earthquake_data_path = dataset_path + "/earthquakes.csv"

datasets = {
    "iris": pd.read_csv(iris_dataset_path),
    "ai_global_index": pd.read_csv(ai_global_index_path),
    "global_earthquake": pd.read_csv(global_earthquake_data_path)
}


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
iris_df = pd.read_csv(iris_dataset_path)
ai_global_index_df = pd.read_csv(ai_global_index_path)
global_earthquake_data_df = pd.read_csv(global_earthquake_data_path)

### BIRCH Implementation (Based on our Algorithm - see report/Part-1.pdf)

In [5]:
class Cluster:
    def __init__(self, points, id):
        self.points = np.array(points)  # Ensure points are stored as a numpy array
        self.id = id  # Unique identifier for the cluster

    def merge(self, other_cluster):
        return Cluster(np.vstack([self.points, other_cluster.points]), self.id)

    def centroid(self):
        return np.mean(self.points, axis=0)


def compute_distance(cluster1, cluster2, linkage='single'):
    if linkage == 'single':
        return min(euclidean(p1, p2) for p1 in cluster1.points for p2 in cluster2.points)
    elif linkage == 'complete':
        return max(euclidean(p1, p2) for p1 in cluster1.points for p2 in cluster2.points)
    elif linkage == 'average':
        distances = [euclidean(p1, p2) for p1 in cluster1.points for p2 in cluster2.points]
        return np.mean(distances)
    else:
        raise ValueError(f"Invalid linkage method: {linkage}")


def hac_custom(data, k, linkage='single'):
    n = len(data)
    clusters = [Cluster([point], i) for i, point in enumerate(data)]
    cluster_ids = [cluster.id for cluster in clusters]

    # Initialize distance matrix
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i + 1, n):
            dist_matrix[i, j] = compute_distance(clusters[i], clusters[j], linkage)
            dist_matrix[j, i] = dist_matrix[i, j]  # Symmetric matrix

    # Use a priority queue (min-heap) to efficiently find the closest clusters
    heap = []
    for i in range(n):
        for j in range(i + 1, n):
            heapq.heappush(heap, (dist_matrix[i, j], i, j))

    # Track merge history for dendrogram (optional)
    merge_history = []

    while len(clusters) > k:
        # Find the closest clusters
        while True:
            min_distance, i, j = heapq.heappop(heap)
            if i in cluster_ids and j in cluster_ids:
                break

        # Merge the closest clusters
        cluster1 = clusters[cluster_ids.index(i)]
        cluster2 = clusters[cluster_ids.index(j)]
        new_cluster = cluster1.merge(cluster2)
        new_cluster_id = new_cluster.id

        # Update clusters and cluster_ids
        clusters = [c for c in clusters if c.id not in (i, j)]
        clusters.append(new_cluster)
        cluster_ids = [c.id for c in clusters]

        # Update distance matrix for the new cluster
        new_distances = []
        for idx, cluster in enumerate(clusters[:-1]):
            new_distance = compute_distance(cluster, new_cluster, linkage)
            new_distances.append(new_distance)
            heapq.heappush(heap, (new_distance, cluster.id, new_cluster_id))

        # Record merge history (optional)
        merge_history.append((i, j, min_distance))

    # Assign labels based on cluster membership
    labels = np.zeros(len(data), dtype=int)
    for cluster_idx, cluster in enumerate(clusters):
        for point in cluster.points:
            point_index = np.where((data == point).all(axis=1))[0]
            if len(point_index) > 0:
                labels[point_index[0]] = cluster_idx

    return labels


if __name__ == "__main__":
    # Sample dataset
    data = np.array([
        [1, 2], [1, 4], [1, 0],
        [10, 2], [10, 4], [10, 0]
    ])

    # Number of clusters
    k = 2

    # Run custom HAC
    labels = hac_custom(data, k, linkage='average')  # Only labels are returned

    print("Cluster Labels:", labels)


Cluster Labels: [0 0 0 1 1 1]


In [8]:
def find_optimum_metrics(datasets):
    results = {}

    for name, df in datasets.items():
        print(f"Dataset: {name}, Type: {type(df)}")

        df = df.dropna()

        # Extract numerical features
        X = df.select_dtypes(include=[np.number]).values

        # Normalize the data
        X = StandardScaler().fit_transform(X)

        # Define parameter grid
        param_grid = {
            'n_clusters': [2, 3, 4],  # Adjust as needed
            'linkage': ['single', 'complete', 'average'],  # Adjust as needed
        }

        # Initialize variables to store the best parameters and metrics
        best_params = None
        best_metrics = {
            'Silhouette Score': -1,
            'Davies-Bouldin Index': float('inf'),
            'Calinski-Harabasz Index': -1,
            'Adjusted Rand Index (ARI)': -1,
            'Homogeneity': -1,
            'Completeness': -1,
            'V-Measure': -1,
        }

        # Iterate over all combinations of parameters
        for n_clusters, linkage in product(param_grid['n_clusters'], param_grid['linkage']):
            print(f"Testing n_clusters={n_clusters}, linkage={linkage}")

            # Run custom Agglomerative Clustering
            try:
                custom_labels = hac_custom(X, n_clusters, linkage)  # Only labels are returned
            except Exception as e:
                print(f"Error in custom Agglomerative Clustering for {name}: {e}")
                continue

            # Run Sklearn Agglomerative Clustering for comparison
            try:
                sklearn_agglo = SklearnAgglomerative(n_clusters=n_clusters, linkage=linkage)
                sklearn_labels = sklearn_agglo.fit_predict(X)
            except Exception as e:
                print(f"Error in Sklearn Agglomerative Clustering for {name}: {e}")
                continue

            # Evaluate clustering using multiple metrics for CUSTOM Agglomerative Clustering
            custom_metrics = {
                'Silhouette Score': silhouette_score(X, custom_labels),
                'Davies-Bouldin Index': davies_bouldin_score(X, custom_labels),
                'Calinski-Harabasz Index': calinski_harabasz_score(X, custom_labels),
                'Adjusted Rand Index (ARI)': adjusted_rand_score(custom_labels, sklearn_labels),
                'Homogeneity': homogeneity_score(sklearn_labels, custom_labels),
                'Completeness': completeness_score(sklearn_labels, custom_labels),
                'V-Measure': v_measure_score(sklearn_labels, custom_labels),
            }

            # Evaluate clustering using multiple metrics for SKLEARN Agglomerative Clustering
            sklearn_metrics = {
                'Silhouette Score': silhouette_score(X, sklearn_labels),
                'Davies-Bouldin Index': davies_bouldin_score(X, sklearn_labels),
                'Calinski-Harabasz Index': calinski_harabasz_score(X, sklearn_labels),
                'Adjusted Rand Index (ARI)': adjusted_rand_score(sklearn_labels, sklearn_labels),
                'Homogeneity': homogeneity_score(sklearn_labels, sklearn_labels),
                'Completeness': completeness_score(sklearn_labels, sklearn_labels),
                'V-Measure': v_measure_score(sklearn_labels, sklearn_labels),
            }

            # Update the best parameters and metrics if the current metrics are better
            if custom_metrics['Silhouette Score'] > best_metrics['Silhouette Score']:
                best_params = {'n_clusters': n_clusters, 'linkage': linkage}
                best_metrics = custom_metrics

            print(f"Metrics: {custom_metrics}")

        # Store the best parameters and metrics for this dataset
        results[name] = {
            'Best Parameters': best_params,
            'Best Metrics': best_metrics,
        }

        # Print the best parameters and metrics for this dataset
        print(f"\nBest Parameters for {name}: {best_params}")
        print(f"Best Metrics for {name}: {best_metrics}")
        print("-" * 50)

    # Save results to a CSV file
    results_df = pd.DataFrame.from_dict({(i, j): results[i][j] 
                                        for i in results.keys() 
                                        for j in results[i].keys()}, 
                                       orient='index')
    results_df.to_csv("./../results/optimum_metrics.csv")

    return results
results = find_optimum_metrics({'iris': datasets.get('iris')})

Dataset: iris, Type: <class 'pandas.core.frame.DataFrame'>
Testing n_clusters=2, linkage=single
Metrics: {'Silhouette Score': 0.554295982740022, 'Davies-Bouldin Index': 0.6208698585960091, 'Calinski-Harabasz Index': 215.7052309869769, 'Adjusted Rand Index (ARI)': 0.9463261709812779, 'Homogeneity': 0.8968976708307189, 'Completeness': 0.9106969052299235, 'V-Measure': 0.9037446161335015}
Testing n_clusters=2, linkage=complete
Metrics: {'Silhouette Score': 0.3725075642289735, 'Davies-Bouldin Index': 1.0854814725646726, 'Calinski-Harabasz Index': 101.17257274441651, 'Adjusted Rand Index (ARI)': 0.6162775009772758, 'Homogeneity': 0.5104148072213296, 'Completeness': 0.5112023543593543, 'V-Measure': 0.5108082772370551}
Testing n_clusters=2, linkage=average
Metrics: {'Silhouette Score': 0.558164122034982, 'Davies-Bouldin Index': 0.6277342636151994, 'Calinski-Harabasz Index': 218.64763137110606, 'Adjusted Rand Index (ARI)': 0.9464323886639676, 'Homogeneity': 0.8900039242634512, 'Completeness': 0

In [9]:
results = find_optimum_metrics({'ai_global_index': datasets.get('ai_global_index')})

Dataset: ai_global_index, Type: <class 'pandas.core.frame.DataFrame'>
Testing n_clusters=2, linkage=single
Metrics: {'Silhouette Score': 0.7358407911622495, 'Davies-Bouldin Index': 0.1730858956119127, 'Calinski-Harabasz Index': 25.94792255616781, 'Adjusted Rand Index (ARI)': 1.0, 'Homogeneity': 1.0, 'Completeness': 1.0, 'V-Measure': 1.0}
Testing n_clusters=2, linkage=complete
Metrics: {'Silhouette Score': 0.7358407911622495, 'Davies-Bouldin Index': 0.1730858956119127, 'Calinski-Harabasz Index': 25.94792255616781, 'Adjusted Rand Index (ARI)': 0.6479240364922734, 'Homogeneity': 0.4224762998723958, 'Completeness': 0.7291878731870094, 'V-Measure': 0.5349903240586071}
Testing n_clusters=2, linkage=average
Metrics: {'Silhouette Score': 0.7358407911622495, 'Davies-Bouldin Index': 0.1730858956119127, 'Calinski-Harabasz Index': 25.94792255616781, 'Adjusted Rand Index (ARI)': 1.0, 'Homogeneity': 1.0, 'Completeness': 1.0, 'V-Measure': 1.0}
Testing n_clusters=3, linkage=single
Metrics: {'Silhouett

In [10]:
results = find_optimum_metrics({'global_earthquake': datasets.get('global_earthquake')})

Dataset: global_earthquake, Type: <class 'pandas.core.frame.DataFrame'>
Testing n_clusters=2, linkage=single
Metrics: {'Silhouette Score': -0.016505019319068844, 'Davies-Bouldin Index': 8.044666961608051, 'Calinski-Harabasz Index': 0.8447911202843066, 'Adjusted Rand Index (ARI)': 0.005694936124365091, 'Homogeneity': 0.1524210492917446, 'Completeness': 0.016950164888053815, 'V-Measure': 0.030507686095497102}
Testing n_clusters=2, linkage=complete
Metrics: {'Silhouette Score': -0.01510202273855693, 'Davies-Bouldin Index': 6.780206918339515, 'Calinski-Harabasz Index': 1.1952233573772715, 'Adjusted Rand Index (ARI)': 0.005694936124365091, 'Homogeneity': 0.1524210492917446, 'Completeness': 0.016950164888053815, 'V-Measure': 0.030507686095497102}
Testing n_clusters=2, linkage=average
Metrics: {'Silhouette Score': -0.01510202273855693, 'Davies-Bouldin Index': 6.780206918339515, 'Calinski-Harabasz Index': 1.1952233573772715, 'Adjusted Rand Index (ARI)': -0.004771432969062644, 'Homogeneity': 0.

In [12]:
# Define the comparison function
def compare_agglo_implementations(datasets, best_parameters):
    all_results = {}

    for name, df in datasets.items():
        print(f"Dataset: {name}, Type: {type(df)}")

        df = df.dropna()

        # Extract numerical features
        X = df.select_dtypes(include=[np.number]).values

        # Normalize the data
        X = StandardScaler().fit_transform(X)

        # Get the best parameters for this dataset
        n_clusters = best_parameters[name]['n_clusters']
        linkage = best_parameters[name]['linkage']

        # Run the custom Agglomerative Clustering implementation
        try:
            custom_labels = hac_custom(X, n_clusters, linkage)
            print(f"Custom Agglomerative Clustering Labels for {name}: {custom_labels}")
        except Exception as e:
            print(f"Error in custom Agglomerative Clustering for {name}: {e}")
            continue

        # Run the Sklearn Agglomerative Clustering implementation
        try:
            sklearn_agglo = SklearnAgglomerative(n_clusters=n_clusters, linkage=linkage)
            sklearn_labels = sklearn_agglo.fit_predict(X)
            print(f"Sklearn Agglomerative Clustering Labels for {name}: {sklearn_labels}")
        except Exception as e:
            print(f"Error in Sklearn Agglomerative Clustering for {name}: {e}")
            continue

        # Evaluate clustering using multiple metrics for CUSTOM Agglomerative Clustering
        custom_metrics = {
            "Adjusted Rand Index (ARI)": adjusted_rand_score(custom_labels, sklearn_labels),
            "Silhouette Score": silhouette_score(X, custom_labels),
            "Davies-Bouldin Index": davies_bouldin_score(X, custom_labels),
            "Calinski-Harabasz Index": calinski_harabasz_score(X, custom_labels),
            "Homogeneity": homogeneity_score(sklearn_labels, custom_labels),
            "Completeness": completeness_score(sklearn_labels, custom_labels),
            "V-Measure": v_measure_score(sklearn_labels, custom_labels),
        }

        # Evaluate clustering using multiple metrics for SKLEARN Agglomerative Clustering
        sklearn_metrics = {
            "Adjusted Rand Index (ARI)": adjusted_rand_score(sklearn_labels, sklearn_labels),
            "Silhouette Score": silhouette_score(X, sklearn_labels),
            "Davies-Bouldin Index": davies_bouldin_score(X, sklearn_labels),
            "Calinski-Harabasz Index": calinski_harabasz_score(X, sklearn_labels),
            "Homogeneity": homogeneity_score(sklearn_labels, sklearn_labels),
            "Completeness": completeness_score(sklearn_labels, sklearn_labels),
            "V-Measure": v_measure_score(sklearn_labels, sklearn_labels),
        }

        # Store results for both implementations
        all_results[name] = {
            "Custom Agglomerative": custom_metrics,
            "Sklearn Agglomerative": sklearn_metrics,
        }

        # Print metrics for both implementations
        print(f"\nMetrics for {name}:")
        print("Custom Agglomerative Metrics:")
        for metric_name, value in custom_metrics.items():
            print(f"{metric_name}: {value}")

        print("\nSklearn Agglomerative Metrics:")
        for metric_name, value in sklearn_metrics.items():
            print(f"{metric_name}: {value}")

    # Store all results in a structured format
    results_df = pd.DataFrame.from_dict(
        {(i, j): all_results[i][j] for i in all_results.keys() for j in all_results[i].keys()},
        orient='index'
    )
    results_df.to_csv("./../results/agglo_comparison_metrics.csv")

    return all_results


# Example Usage
if __name__ == "__main__":
    # Load your datasets
    from sklearn.datasets import load_iris

    iris = load_iris()
    iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)

    # Define the best parameters for each dataset
    best_parameters = {
        'iris': {'n_clusters': 2, 'linkage': 'average'},
        'ai_global_index': {'n_clusters': 2, 'linkage': 'single'},
        'global_earthquake': {'n_clusters': 4, 'linkage': 'average'},
    }

    
    # Compare custom Agglomerative and Sklearn Agglomerative for all datasets
    results = compare_agglo_implementations(datasets, best_parameters)

Dataset: iris, Type: <class 'pandas.core.frame.DataFrame'>
Custom Agglomerative Clustering Labels for iris: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
 1 1]
Sklearn Agglomerative Clustering Labels for iris: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0]

Metrics for iris:
Custom Agglomerative Metrics:
Adjusted Rand Index (ARI): 0.9464323886639676
Silhouette Score: 0.558164122034982
Davies-Bouldin Index: 0.6277342636151994
Calinski-Harabasz Index: 218.64763137110606
Homogeneity: 0